# Overview
This notebook is the first notebook in a the SQL series where I document and track my progress learning SQL.

## Basics
### Select ++ 
* for a database db, one can store the results of a query as follows: CREATE VIEW view_name AS QUERY;
* The result can be further accessed as follows: SELECT * FROM view_name;
* Count(field_name) from db: returns the number of records containing a value in that field
* Count(DISTINCT field_name) from db: returns the number of records containing a value in that field
* SELECT COUNT(field) as alias from db:
if the name of a field contains spaces it should be enclosed in "". for example, "release year"
* KEEP in mind the difference between single and double quotes: 
    1. "" are used to refer to database objects usually: tables, columns or even functions (When the function name is a reserved keyword)
    2. '' are used of string literals: they are evaluated as they are


### Filtering: where
* SELECT col_names from table_name where condition  
* Conditions can use arithmetic operators such as '=' for equa, and <> for not equal.
* WHERE col <= a and col >= b  $\iff$ WHERE col between a AND b 
* We can use the following to filter text data:
    1. _ matches a single character, % match any number of characters.
* We can use ***IN*** (list of values seperated by ','): works as IN in the python language.
* the **LINK** logical operator is case-sensitive. The case non-sensitive version is the logical operator **ILIKE**



#### NULL
Databases are related to Data and NO data without missing values: Here are some key points:

1. COUNT(*): includes missing values
2. COUNT(field) will return only the rows where the field in question is not NULL 
3. IS (NOT) NULL can be used to filter values.

#### Aggregating Functions
SQL offers additional tools to better understand the data. AVG(col_name), MIN(col_name), MAX(col_name), SUM(col_name), COUNT(col_name)
apply statistics on the column in question. Filtering can be used as well.   
* AVG and SUM work only on numerical data type while the rest work also on non-numerical data.
* columns created using aggregating functions or arithmetic operatations: +, -, /, * must be aliased for further use.


### Sorting
Sorting is quite important for many tasks:
* ORDER BY AT THEN END: ORDER BY col1 ASC/DESC, col2 ASC /DESC, ...
### Aggregating : aggregate functions
* COUNT function returns the number of non-null values in a certain column.
* COUNT(*) / COUNT(1) can be used to evaluate the number of rows in a table
* SUM treats nulls are $0$ 's
* MIN and MAX functions work for any table of data that supports sorting.
* One important remark about the ***AVG*** function is that it ignores null values completely: in other words, the following two queries will return the exact same results:  
SELECT AVG(column) FROM TABLE;  
SELECT AVG(column) FROM TABLE where column NOT NULL;


### Aggregating: Group BY
* I would like to think of **GROUP BY** as follows: this command creates a number of temporary tables internally where each table have a unique value of the column (or combination of columns) used to group. 
* Having this in mind, the query can return any of the fields used for grouping, or the result of an aggregating function applied over the values of the other fields in each temporary table. 
* IF a filtering condition is set on a result of aggregate function, it requires the use of the keyword HAVING.
* THIS is mainly explained by the order of execution as FROM, WHERE, GROUP BY, HAVING: so WHERE is called before the groups are created and thus no filtering based on the results of aggregation can be made.

* The following question poses itself: how to filter the aggregated results. Well the **WHERE** clause would not work here. This is why we use its equivalent clause **HAVING**. The latter can filter the results by two means:
    1. operators on one of the columns used for grouing as they are integrated in the internal temporary tables
    2. operators on an aggregate functions on any column 
     
These limitations are imposed due to the same reasons explained above with the SELECT statement.

### Case statement
* The case statement is quite a powerful tool as it can be used in very creative ways.
* The case's statement can be found through this [link](https://mode.com/sql-tutorial/sql-case/) 

### Distinct
* Well the **DISTINCT** keyword is usually associated with aggregation.
* SELECT DISTINCT col_1, col_2 ..., col_n FROM table: returns each unique value in the table in a separate row. 
* THE SELECT DISTINCT col_1, DISTINCT col_2 does not compile when the number of distinct values in the two columns differ. (which makes sense as SQL works only with non-ragged tables)
* The main application of DISTINCT is with the COUNT clause: getting the number of unique values per groups.
* It is important to note that DISTINCT will sort the data internally (simple DSA knowledge), thus it slows the query's performance significantly.

* ***FINAL NOTE***: the combination of aggregate functions and usual columns requires the use of the ***GROUP BY*** clause.

## JOINING TABLES

* Joining tables is used to create larger tables out of other existent tables:   
SELECT table.common\_field, table.common\_field, field\_1, field\_2  
FROM table1  
inner JOIN  table\_2  
ON table\_field = table\_field AND table\_field\_1 = table\_field\_2

* if the field we are joining on is common between tables we can use the expression: ***USING(common\_field)***
* if all columns froms a certain table are needed then they can be selected as follows, table_name.*

* Inner join receives two tables and creates a new table with only the rows that share common values for the chosen fields.
* LEFT join receives two tables and creates a new table with the original rows of the left table and the columsn of the right table. Fields with common chosen fields will have non-null values while the others will have null-values.
* 

* filtering during joining tables can be done either with **ON** statement or **WHERE**. The two options are not equivalent though
* the **ON** filtering condition is applied only on the table associated with the JOIN clause. On the other hand, The **WHERE** filtering condition is applied on the resulting table. The first option might lead to performance gains as **joining** is more costly than **filtering**.


## Appending Tables
Joining tables work with columns (vertically). Nevertheless, SQL users would probably to manipulate their tables horizontally, row-wise. **UNION** and **UNION ALL** will do the trick:  

SELECT STATEMENT 1  
UNION /  UNION ALL  
SELECT STATEMENT 2  

TWO TABLES CAN BE JOINED if they satisfy two condititions:  
1. they have the same number of columns
2. they have the same data types in respective columns  

The difference between **UNION** and **UNION ALL** is that the latter will keep duplicate rows present in the first table. **UNION** will keep only the unique rows.

## Sub-queries
* we can use subqueries in different scenarios. The simplest is probably with the **FROM** clause.
* each subquery should have its own alias, wrapped between parentheses
* subqueries might be the most efficient and probably the only way to solve certain problems such as aggregation in multiple stages. This is greatly explained by the [MODE tutorial](https://mode.com/sql-tutorial/sql-sub-queries/)
* subqueries with the FROM clause has no restrictions
* subqueries with the WHERE and most logical operators must be one-cell value. The ***IN*** operator can use multiple values (either rows or columns)
* subqueries with logical operators should not be associated with an alias as they are treated either as  single value or a set of values.

## window functions
so let's start with a definition (a surprisingly expressive one!!). SO as Postgre's documentation states, a window function is a functionality that carries out the same type of calculations as aggregate functions while having the rows separated instead of being meshed into a single output as the aggregate functions do.  
Let's delve into more details while adding some more examples.

In [1]:
# let's first import the sql connection utilities written on a different python file
from mysql_python import connect_config
from mysql_python import write_db_config


config_file_path = write_db_config(host="localhost",database='sql_zoo', user='ayhem_vscode', password='Imadeyoumy97*')
connection = connect_config(config_file_path)

Connection established.


In [ ]:
! pip install pymysql

In [13]:
! pip install sqlalchemy
# also know as MySQLdb
! pip install mysqlclient 

     ------------------------------------ 178.5/178.5 kB 978.4 kB/s eta 0:00:00


In [18]:
import pandas as pd
from mysql_python import read_db_config
from sqlalchemy import create_engine

config_dict = read_db_config("ayhem_vscode_localhost_sql_zoo.ini")

url = f"mysql://{config_dict['user']}:{config_dict['password']}@{config_dict['host']}/{config_dict['database']}"
connection = create_engine(url, echo=False).connect()

# let's try out our new connection
pd.read_sql_query("SELECT * FROM employee LIMIT 10", con=connection)


,id,name,SALARY,DEPARTMENTID
0,1,a,40,1
1,2,b,45,1
2,3,c,50,1
3,4,d,50,1
4,5,da,60,1
5,6,ds,60,1
6,7,dc,85,1
7,8,de,90,1
8,9,x,60,2
9,10,x1,65,2


In [ ]:
# here is a usual group by  SQL statement
q1 = pd.read_sql_query("SELECT departmentid, AVG(salary) FROM employee GROUP BY DEPARTMENTID;", con=connection)
print(q1)
# neverthess here the same query with windowns functions
q2 = pd.read_sql_query("SELECT departmentid, name, salary, AVG(salary) OVER (PARTITION BY departmentid) as average_salary FROM employee;", con=connection)
print(q2)
# the 2nd query shows that the same calculations are conducted without grouping all the rows with common field value into a single row.
# these are the window functions at their most basic form


* The general syntax of a window function:  

SELECT fileds, function_name(expression) OVER (  
\[partition definition\]  
\[order_definition \]  
\[frame definition\]  
)  
* the parentheses of the OVER clause are mandatory regardless of its inner components
* the partition clause breaks the table into parts. The partition function is performed over these chuncks of rows and re-initialized at the start of each one.
* the order clause determines the rows' order within each partition. Multiple keys can be used to set an order for the rows as all window functions support the order. Nevertheless, it makes sense to use the latter only for order-sensitive functions.
*  

## More functions 
* casting to different data types in mysql: [link](https://www.w3schools.com/sql/func_mysql_cast.asp)
* 